In [1]:
mes = "09"

In [2]:
#manipulação de dados
import pandas as pd
import numpy as np

#automação
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

#web scraping
import requests
from bs4 import BeautifulSoup

#string e data
import unidecode

#tempo
import time
from time import sleep
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.expected_conditions import (visibility_of, 
                                                           staleness_of, 
                                                           invisibility_of_element)

import warnings #Warning inuteis
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

import unidecode #arrumar strings

import re #regex
import os


#pacotes para manipular data
from datetime import date
from datetime import time
from datetime import datetime
from datetime import timedelta

# Automação

In [3]:
browser = webdriver.Firefox()

browser.get('https://www.pegaplantao.com.br/login')
sleep(1)
browser.find_element("xpath", '//*[@id="MainContent_LoginUser_UserName"]').send_keys('gabriel_s_anjos@yahoo.com') #e-mail
browser.find_element("xpath", '//*[@id="MainContent_LoginUser_Password"]').send_keys('Gabriel2022!') #senha
browser.find_element("xpath", '//*[@id="MainContent_LoginUser_btnLogin"]').click() #login

In [4]:
browser.get('https://www.pegaplantao.com.br/EscalaMensal')

In [5]:
#click setor/profissional
browser.find_element("xpath", '//*[@id="s2id_autogen1"]/a/span[1]').click()
#click [profissional]
browser.find_element("xpath", '//*[@id="select2-drop"]/ul/li[2]/div').click()

# Capturando setores:

In [6]:
#click criar novo plantão
browser.find_element("xpath", '//*[@id="shift-monthly"]/div[2]/table/tbody/tr[1]/td[4]/button').click()

In [7]:
import time
from time import sleep

In [8]:
inicio = time.time() #iniciando o cronometro

#Setor
browser.find_element("xpath", '//*[@id="s2id_ddlServiceSector"]/a/span[1]').click()

setor_html = browser.find_element("xpath", '//*[@id="select2-drop"]/ul').get_attribute('outerHTML')
soup = BeautifulSoup(setor_html, 'html.parser')

fim = time.time() #encerrando cronometro
print('\nTempo que demorou o codigo:', fim - inicio)


Tempo que demorou o codigo: 0.764646053314209


In [9]:
inicio = time.time() #iniciando o cronometro

lista_unidade = []
lista_unidade_count = []
lista_plt = []
lista_plt_count = []

count_unidade = 1

for i in soup.find_all(class_="select2-results-dept-0 select2-result select2-result-unselectable select2-result-with-children"):
    #print('\n', i.find(class_='select2-result-label').get_text(), '====== \n')
    count_plt = 1
    for t in i.find_all(class_='select2-results-dept-1 select2-result select2-result-selectable'): 
        #print(t.get_text())
        lista_plt.append(t.get_text())
        lista_plt_count.append(count_plt)
        count_plt += 1
            
        lista_unidade.append(i.find(class_='select2-result-label').get_text())
        lista_unidade_count.append(count_unidade)
    count_unidade += 1

scrap_unidades = pd.DataFrame({'unidade':lista_unidade, 'momento':lista_plt,
                               'unidade_count':lista_unidade_count, 'plt_count':lista_plt_count})

scrap_unidades['momento'] = scrap_unidades['momento'].apply(lambda x: x.replace('  ',' '))

scrap_unidades['momento'] = scrap_unidades['momento'].apply(lambda x: unidecode.unidecode(x.replace('  ',' ').upper().strip()))
scrap_unidades['unidade'] = scrap_unidades['unidade'].apply(lambda x: unidecode.unidecode(x.replace('  ',' ').upper().strip()))

fim = time.time() #encerrando cronometro
print('\nTempo que demorou o codigo:', fim - inicio)




Tempo que demorou o codigo: 0.013543844223022461


# TESTANDO SETORES

In [10]:
df_backup = pd.read_excel('df_giov2.xlsx')

df_backup = df_backup.sort_values(['nome', 'data'])
df_backup.reset_index(drop=True, inplace=True)

#df_backup['periodo'] = df_backup['periodo2']

#FATIAMENTO NO DATAFRAME
df_backup_up = df_backup.copy()

print(len(df_backup))
df_backup.sample(2)

7


,chat,dia,nome,mensagem,controle,data,unidade,extra,tipo,ENTRADA,SAIDA,DURAÇÃO,PERIODO,periodo,setor,sem_ou_fds,unidade2,setor_inicio
4,[29/09/2023 17:26:16] Pedro Henrique Diniz Bot...,29/09/2023,PEDRO HENRIQUE DINIZ BOTELHO,29/09 Anália - Diniz saída 17:22 ( pós - ...,PROB,29/09/2023,ANALIA,NaN,SAIDA,13:00,17:30,04:30,tarde,3,SEMANA - TARDE - C.CIRURGICO,semana,"['ANALIA', 'MORUMBI']",SEMANA - TARDE
2,[29/09/2023 15:27:35] Marcela Aparecida Correa...,29/09/2023,MARCELA APARECIDA CORREA MARTINS,29/9 Alpha - Martins entrada 07:00 ( desl...,PROB,29/09/2023,ALPHA,NaN,ENTRADA,07:00,13:00,06:00,manha,2,SEMANA - MANHA - C.CIRURGICO,semana,"['ALPHA', 'ANALIA']",SEMANA - MANHA


In [11]:
df_backup['PERIODO'].unique()

array(['tarde', 'manha'], dtype=object)

In [12]:
df_backup['PERIODO'].unique()

df_backup['periodo'] = np.nan
df_backup['periodo'][df_backup['PERIODO'] == '6h'] = 1
df_backup['periodo'][df_backup['PERIODO'] == 'manha'] = 2
df_backup['periodo'][df_backup['PERIODO'] == 'tarde'] = 3
df_backup['periodo'][df_backup['PERIODO'] == 'noturno saida'] = 4

In [13]:
print(df_backup['setor'].unique())

['SEMANA - TARDE - C.CIRURGICO' 'SEMANA - MANHA - C.CIRURGICO']


In [14]:
print(df_backup['setor'].unique())
df_backup['setor_inicio'] = df_backup['setor']
df_backup['setor'][df_backup['setor'] == 'SEMANA - MANHA'] = 'SEMANA - MANHA - C.CIRURGICO'
df_backup['setor'][df_backup['setor'] == 'SEMANA - TARDE'] = 'SEMANA - TARDE - C.CIRURGICO'

['SEMANA - TARDE - C.CIRURGICO' 'SEMANA - MANHA - C.CIRURGICO']


In [15]:
print(len(df_backup))
df_backup_fds = df_backup[(df_backup['sem_ou_fds'] != 'semana') | (df_backup['tipo'] != 'SAIDA')].reset_index(drop=True)
print(len(df_backup_fds))
df_backup_up = df_backup[(df_backup['sem_ou_fds'] == 'semana') & (df_backup['tipo'] == 'SAIDA')].reset_index(drop=True)
print(len(df_backup_up))

df = df_backup_up.copy()

7
2
5


In [16]:
df_backup_up

,chat,dia,nome,mensagem,controle,data,unidade,extra,tipo,ENTRADA,SAIDA,DURAÇÃO,PERIODO,periodo,setor,sem_ou_fds,unidade2,setor_inicio
0,[29/09/2023 16:51:51] Débora Philippi Bressane...,29/09/2023,DEBORA PHILIPPI BRESSANE,29/09 Jabaquara Débora Bressane saída 16:40 ...,PROB,29/09/2023,JABAQUARA,NaN,SAIDA,13:00,17:00,04:00,tarde,3.0,SEMANA - TARDE - C.CIRURGICO,semana,JABA,SEMANA - TARDE - C.CIRURGICO
1,[29/09/2023 13:09:48] Guilherme de Almeida Cos...,29/09/2023,GUILHERME DE ALMEIDA COSTA,29/09 osasco costa saida 13:09 ( pos planta...,PROB,29/09/2023,OSASCO,NaN,SAIDA,13:00,13:30,00:30,tarde,3.0,SEMANA - TARDE - C.CIRURGICO,semana,"['OSASCO', 'ITAIM']",SEMANA - TARDE - C.CIRURGICO
2,[29/09/2023 15:28:27] Marcela Aparecida Correa...,29/09/2023,MARCELA APARECIDA CORREA MARTINS,29/09 Alpha - Martins saída 15:00 ( deslo...,PROB,29/09/2023,ALPHA,NaN,SAIDA,13:00,15:00,02:00,tarde,3.0,SEMANA - TARDE - C.CIRURGICO,semana,"['ALPHA', 'ANALIA']",SEMANA - TARDE - C.CIRURGICO
3,[29/09/2023 17:26:16] Pedro Henrique Diniz Bot...,29/09/2023,PEDRO HENRIQUE DINIZ BOTELHO,29/09 Anália - Diniz saída 17:22 ( pós - ...,PROB,29/09/2023,ANALIA,NaN,SAIDA,13:00,17:30,04:30,tarde,3.0,SEMANA - TARDE - C.CIRURGICO,semana,"['ANALIA', 'MORUMBI']",SEMANA - TARDE - C.CIRURGICO
4,[29/09/2023 14:37:19] Rodrigo Miyashiro Nunes ...,29/09/2023,RODRIGO MIYASHIRO NUNES DOS SANTOS,29/09 MORUMBI - Miyashiro saida 14:35 ( d...,PROB,29/09/2023,MORUMBI,NaN,SAIDA,13:00,15:00,02:00,tarde,3.0,SEMANA - TARDE - C.CIRURGICO,semana,"['MORUMBI', 'JABA']",SEMANA - TARDE - C.CIRURGICO


In [17]:
for X in range(0, len(df)):
    if len(scrap_unidades.loc[(scrap_unidades.unidade == df['unidade'][X]) & 
                                 (scrap_unidades.momento == df['setor'][X])]) < 1:
        print(X)
        print(df['unidade'][X], '-', df['setor'][X])
        #print('index', ind)
        #print(scrap_unidades['unidade'][ind], '-', scrap_unidades['momento'][ind])
        #print('codigos:', unidade, setorzin)
    #else:
        #print('='*100, df['unidade'][X], '-', df['setor'][X])

print("="*20, "TUDO OK", "="*20)

==================== TUDO OK ====================


# Função de captura dos dados

In [18]:
#########2
#capturando registros
unidades = ['ADMINISTRACAO','ALPHA','ANALIA','AVICCENA',
            'ITAIM','JABAQUARA','MATERNIDADE','MORUMBI',
            'OSASCO','VILLA','VNS','SANTA ISABEL']

def funcao():
    lista_tabela = []
    for i in range(1, 6): #passando por cada semana[1 a 5]
    ##print('semana:', i, "=="*20)
        for t in range(1, 8): #passando por cada dia[1 a 7] dentro da semana
            reg = browser.find_element("xpath", '/html/body/form/main/div[2]/div[2]/table/tbody/tr['+str(i)+']/td['+str(t)+']').text
            content = reg.splitlines() #deixando as palavras em um item dentro de uma lista
            content = [unidecode.unidecode(i.strip().upper()) for i in content] #tratando cada palavra

            dia = content[0] #dia do registro
            del(content[0]) #deletando o dia, ja que foi coletado
            del(content[-1]) #deletando resido do wepscraping

            if len(content) != 0: #ignorando dias em branco (sem registro) ¹
                #print('dia:', t)

                ##print('dia:', dia)
                #print(content)

                count_hora = 0 #count para capturar a hora de entrada e saida, count=1(entrada) | count=2(saida)
                lista_control = []
                for y in content: #passando um for sobre cada item da lista
                    if y in unidades: # capturando a unidade e dando inicio a novo registro novo
                        #print('================unidade:', y)
                        unidade = y
                    elif y[2] == ':': #reconhecendo se o item é hora (HH*:*mm)
                        count_hora += 1
                        if count_hora == 1: #capturando entrada
                            #print('hora entrada:', y)
                            lista_control.append(y)
                        elif count_hora == 2: #capturando saida
                            #print('hora saida:', y)
                            lista_control.append(y)
                            ##print('lista teste', lista_control)
                            lista_tabela.append(lista_control)
                            lista_control = []
                            count_hora = 0
                    else: #capturando dia, unidade, e periodo
                        #print('.')
                        #print('periodo:', y)
                        lista_control.append(dia)
                        lista_control.append(unidade)
                        lista_control.append(y)
                        lista_control.append(i)
                        lista_control.append(t) #dia semanal

                ##print('='*20)    

            else:
                continue #ignorando dias em branco (sem registro) ²
    
    tabela = pd.DataFrame(lista_tabela, columns = ['Dia','Unidade','Periodo','Semana','Dia Semana','Entrada','Saida'])
    
    #limpeza da tabela
    #########3
    tabela['Dia'] = tabela['Dia'].apply(lambda x: int(x))

    #recolhendo os dias do mes passado que ainda aparecem na primeira semana
    dias_mes_passado = list(tabela[(tabela['Dia'] > 20) & (tabela['Semana'] == 1)].index) 
    tabela.drop(dias_mes_passado, axis=0, inplace=True) #excluindo
    tabela.reset_index(drop=True, inplace=True) #resetando index

    #Inserindo a ordem dos registros
    tabela['ordem'] = ''

    for i in tabela['Dia'].unique():
        for t in range(0, len(tabela[tabela['Dia'] == i].index)):
            tabela['ordem'][tabela[tabela['Dia'] == i].index[t]] = t+1

    #Inserindo coluna de data        
    tabela['data'] = tabela['Dia'].apply(lambda x: str(x))

    for i in range(len(tabela)):
        if len(tabela['data'][i]) == 1:
            tabela['data'][i] = "0" + tabela['data'][i]

        tabela['data'][i] = tabela['data'][i] + "/" + mes + "/2023" 

    tabela = tabela.query("Unidade != 'ADMINISTRACAO'")
    tabela = tabela[~((tabela['Semana'] >= 5) & (tabela['Dia'] < 10))]
    tabela.reset_index(drop=True, inplace=True) #resetando index
    
    tabela['PERIODO'] = np.nan
    for i in range(0, len(tabela)):
        if int(tabela['Entrada'][i][:2]) <= 6:
            tabela['PERIODO'][i] = '6h'
        elif int(tabela['Entrada'][i][:2]) >= 7 and int(tabela['Entrada'][i][:2]) <= 12:
            tabela['PERIODO'][i] = 'manha'
        elif int(tabela['Entrada'][i][:2]) >= 13 and int(tabela['Entrada'][i][:2]) <= 18:
            tabela['PERIODO'][i] = 'tarde'
        else:
            tabela['PERIODO'][i] = 'noturno saida'

    tabela['periodo'] = np.nan
    tabela['periodo'][tabela['PERIODO'] == '6h'] = 1
    tabela['periodo'][tabela['PERIODO'] == 'manha'] = 2
    tabela['periodo'][tabela['PERIODO'] == 'tarde'] = 3
    tabela['periodo'][tabela['PERIODO'] == 'noturno saida'] = 4
    
    
    
    return tabela
    
tabela = funcao()
tabela

,Dia,Unidade,Periodo,Semana,Dia Semana,Entrada,Saida,ordem,data,PERIODO,periodo


# APAGADO:

In [19]:
len(df_backup)

7

In [20]:
count = 0

In [21]:
fdzão = pd.DataFrame(columns = tabela.columns)
fdzão

,Dia,Unidade,Periodo,Semana,Dia Semana,Entrada,Saida,ordem,data,PERIODO,periodo


In [22]:
from selenium.webdriver.common.action_chains import ActionChains

action_chains = ActionChains(browser)

In [26]:
#Para cada profissional na lista
df = df_backup[count:]

for i in df['nome'].unique():
    print("nome:", i)
    df_nome = df[df['nome'] == i]
    df_nome.reset_index(drop=True, inplace=True)
    
    browser.find_element("xpath", '//*[@id="s2id_ddlProfessionals"]/a/span[1]').click()
    #browser.find_element("xpath", '/html/body/form/main/div[2]/div[2]/div/div[1]/div[3]/professionals/div/a/span[2]/b').click()
    sleep(2)
    
    browser.find_element("xpath", '/html/body/div[9]/div/input').send_keys(i)
    browser.find_element("xpath", '//*[@id="select2-drop"]/ul/li/div').click()

    #esperando carregamento do site
    WebDriverWait(browser, 30).until_not(visibility_of(browser.find_element("xpath", '//*[@id="loader"]')))

    sleep(2)
    tabela = funcao()  
    sleep(2)
    
    #Para cada registro na lista_nome
    for t in range(len(df_nome)):
        dia_sel = tabela[(tabela['data'] == df_nome['data'][t]) & (tabela['Unidade'] != df_nome['unidade'][t])]
        dia_sel.reset_index(drop=True, inplace=True)
        
        print(df_nome['chat'][t] + " - " + df_nome['data'][t] + ' - ' + df_nome['unidade'][t] + " - " + df_nome['ENTRADA'][t] + " - " + df_nome['SAIDA'][t])
        #display(dia_sel)
        if len(dia_sel) == 0:
            print(count, "-> PROB")
            df_backup['controle'][count] = "PROB"
        else:
            fd = pd.DataFrame(columns = dia_sel.columns) #df de captura para recriação
            for r in range(len(dia_sel)-1, -1, -1):
                if dia_sel['periodo'][r] <= df_nome['periodo'][t]: #capturando dados
                    print('PRINCIPAL')
                    #abrindo/selecionando periodo conforme localização
                    dia_ordem = r
                    xpath_textao = '//*[@id="shift-monthly"]/div[2]/table/tbody/tr['+str(dia_sel['Semana'][0])+']/td['+str(dia_sel['Dia Semana'][0])+']/ul['+str(dia_sel['ordem'][dia_ordem])+']/li'
                    elemento = browser.find_element("xpath", xpath_textao)
                    elemento.click()

                    #Esperar componentes carregarem
                    WebDriverWait(browser, 10).until(visibility_of(browser.find_element("xpath", '//*[@id="service-ckb-list-all-owner"]')))

                    #Apagando
                    browser.find_element("xpath", '/html/body/div[5]/div/div[1]/div[2]/button[1]').click()
                    #Clicando em continuar
                    sleep(0.5)
                    browser.find_element("xpath", '/html/body/div[5]/div/div[1]/div[3]/button[2]').click()
                    
                    if dia_sel['periodo'][r] < df_nome['periodo'][t]:
                        fd.loc[len(fd)] = list(dia_sel.loc[r])
                    
                    #esperando carregamento do site para prosseguir
                    WebDriverWait(browser, 10).until_not(visibility_of(browser.find_element("xpath", '//*[@id="modal-add-service"]/div/div[1]/button')))

            fd = fd.sort_values(['periodo'])
            fd.reset_index(drop=True, inplace=True)
            fd['Unidade_antiga'] = fd['Unidade']
            fd['Unidade'] = df_nome['unidade'][0]
            fd['nome'] = df_nome['nome'][0]
            fdzão = pd.concat([fdzão, fd]) # juntando ao df total de captura
            fdzão = fdzão.reset_index(drop=True)
            
            tabela = funcao() #atualizando tabela de registros
            sleep(2)
            print(count, '-> OK')
            df_backup['controle'][count] = "OK"
            
        count += 1
            
        if count % 25 == 0:
            print('='*80)
            print(df_backup['controle'].value_counts())
            print('='*80)
            
    print('\n')
print(df_backup['controle'].value_counts())

nome: GUILHERME DE ALMEIDA COSTA
[29/09/2023 13:09:48] Guilherme de Almeida Costa: 29/09 osasco costa saida 13:09 (pos plantao) (deslocado itaim) - 29/09/2023 - OSASCO - 13:00 - 13:30
PRINCIPAL
PRINCIPAL
1 -> OK


nome: MARCELA APARECIDA CORREA MARTINS
[29/09/2023 15:27:35] Marcela Aparecida Correa Martins: 29/9 Alpha - Martins entrada 07:00 (deslocada do Anália) - 29/09/2023 - ALPHA - 07:00 - 13:00
PRINCIPAL
2 -> OK
[29/09/2023 15:28:27] Marcela Aparecida Correa Martins: 29/09 Alpha - Martins saída 15:00 (deslocada do Anália) - 29/09/2023 - ALPHA - 13:00 - 15:00
PRINCIPAL
3 -> OK


nome: PEDRO HENRIQUE DINIZ BOTELHO
[29/09/2023 17:26:16] Pedro Henrique Diniz Botelho: 29/09 Anália - Diniz saída 17:22 (pós-plantão / deslocado do Morumbi) - 29/09/2023 - ANALIA - 13:00 - 17:30
PRINCIPAL
PRINCIPAL
4 -> OK


nome: RICARDO IVO DE LIMA
[29/09/2023 00:28:16] Ricardo Ivo de Lima: 28/09 - Itaim - Ricardo Ivo Entrada - 07:00h (Deslocado Morumbi - cirurgia a pedido) - 28/09/2023 - ITAIM - 07:00 - 

In [28]:
fdzão

,Dia,Unidade,Periodo,Semana,Dia Semana,Entrada,Saida,ordem,data,PERIODO,periodo,Unidade_antiga,nome
0,31,OSASCO,SEMANA - MANHA - C.CIRURGICO,5,4,07:00,13:00,1,31/08/2023,manha,2.0,MATERNIDADE,BEATRIZ ANTIPOU DOS SANTOS
1,28,AVICCENA,SEMANA - MANHA - C.CIRURGICO,5,1,07:00,13:00,1,28/08/2023,manha,2.0,JABAQUARA,EMERSON DOMINGOS DA COSTA
2,29,VNS,SEMANA - MANHA - C.CIRURGICO,5,2,07:00,13:00,1,29/08/2023,manha,2.0,ITAIM,FABIO ANTONIO DE SOUZA OLIVEIRA
3,29,VNS,SEMANA - TARDE - C.CIRURGICO,5,2,13:00,19:00,2,29/08/2023,tarde,3.0,ITAIM,FABIO ANTONIO DE SOUZA OLIVEIRA
4,28,VNS,SEMANA - MANHA - C.CIRURGICO,5,1,07:00,13:00,1,28/08/2023,manha,2.0,ITAIM,FELIPE COELHO DE SOUZA
5,25,ITAIM,SEMANA - MANHA - C.CIRURGICO,4,5,07:00,13:00,1,25/08/2023,manha,2.0,VNS,FERNANDO RIBEIRO SA
6,28,ANALIA,SEMANA - MANHA - C.CIRURGICO,5,1,07:00,13:00,1,28/08/2023,manha,2.0,VILLA,KAREN YUMI MIYASHIRO
7,31,AVICCENA,SEMANA - MANHA - COBERTURA EXTRA,5,4,07:00,13:00,1,31/08/2023,manha,2.0,VILLA,LAYSA LAYNA RIBEIRO LIMA
8,29,OSASCO,SEMANA - MANHA - C.CIRURGICO,5,2,07:00,13:00,2,29/08/2023,manha,2.0,VNS,MAIELY MARCOLIN
9,30,OSASCO,SEMANA - MANHA - C.CIRURGICO,5,3,07:00,13:00,1,30/08/2023,manha,2.0,MATERNIDADE,MAIELY MARCOLIN


In [29]:
count += 1

In [30]:
browser.find_element("xpath", '/html/body/div[6]/div/div[1]/div[2]/button[1]').click()
                                /html/body/div[4]/div/div[1]/div[2]/button[1]

In [36]:
len(df_backup)

10

In [65]:
df_backup['nome'][df_backup['nome'] == 'ICARO MOURA'] = 'ICARO DE MOURA SOUSA'

In [33]:
df_backup['controle'][count] = "PROB"
count += 1

In [34]:
print(df_backup['controle'].value_counts())

OK      11
PROB     2
Name: controle, dtype: int64


In [38]:
fdzão
#fdzão = fdzão.drop([1]).reset_index(drop=True)

,Dia,Unidade,Periodo,Semana,Dia Semana,Entrada,Saida,ordem,data,PERIODO,periodo,Unidade_antiga,nome
0,19,OSASCO,SEMANA - MANHA - C.CIRURGICO,4,3,07:00,13:00,1,19/07/2023,manha,2.0,ITAIM,GUILHERME DE ALMEIDA COSTA
1,20,OSASCO,SEMANA - MANHA - C.CIRURGICO,4,4,07:00,13:00,2,20/07/2023,manha,2.0,VNS,GUSTAVO MARQUES DA SILVA
2,19,JABAQUARA,SEMANA - MANHA - C.CIRURGICO,4,3,07:00,13:00,1,19/07/2023,manha,2.0,MORUMBI,ICARO DE MOURA SOUSA
3,19,VNS,SEMANA - MANHA - AV.PRE ANESTESICA,4,3,07:00,13:00,1,19/07/2023,manha,2.0,MORUMBI,JESSICA LIDIA DE SOUZA
4,19,OSASCO,SEMANA - MANHA - C.CIRURGICO,4,3,07:00,13:00,1,19/07/2023,manha,2.0,ITAIM,LAURA SILVA ASSUNCAO MAPA
5,19,OSASCO,SEMANA - MANHA - COBERTURA EXTRA,4,3,07:00,13:00,2,19/07/2023,manha,2.0,VNS,LAYSA LAYNA RIBEIRO LIMA
6,19,OSASCO,SEMANA - MANHA - C.CIRURGICO,4,3,07:00,13:00,1,19/07/2023,manha,2.0,MORUMBI,MANUELA AFFONSO FERREIRA SANGED
7,19,JABAQUARA,SEMANA - MANHA - COBERTURA EXTRA,4,3,07:00,13:00,2,19/07/2023,manha,2.0,ANALIA,MATHEUS CARDOZO JANIQUES


# ARRUMANDO A PORRA TODA:

In [27]:
df_backup['controle'].value_counts()

OK    7
Name: controle, dtype: int64

In [28]:
df = df_backup[df_backup['controle'] == 'OK'].reset_index(drop=True)
print(len(df))
df_prob = df_backup[df_backup['controle'] != 'OK'].reset_index(drop=True)
print(len(df_prob))

7
0


In [29]:
print(len(fdzão))
fdzão.head(1)

4


,Dia,Unidade,Periodo,Semana,Dia Semana,Entrada,Saida,ordem,data,PERIODO,periodo,Unidade_antiga,nome
0,29,JABAQUARA,SEMANA - MANHA - C.CIRURGICO,5,5,07:00,13:00,1,29/09/2023,manha,2.0,ITAIM,DEBORA PHILIPPI BRESSANE


In [30]:
fdzão = fdzão.rename({'Unidade':'unidade', 'Periodo':'setor',
                     'Entrada':'ENTRADA', 'Saida':'SAIDA'}, axis=1)

In [31]:
#DURAÇÃO DO PLANTÃO
fdzão['DURAÇÃO'] = np.nan

for i in range(0, len(fdzão)):
    diferenca_tempo = datetime.strptime(fdzão['SAIDA'][i], '%H:%M') - datetime.strptime(fdzão['ENTRADA'][i], '%H:%M')
    
    #diferenca_tempo vem em formato estranho, tem-se que transformar em segundos antes
    diferenca_tempo_s = diferenca_tempo.seconds
    horas = '0' + str(int((diferenca_tempo_s/60)/60)) #capturando horas
    minutos = str(int((diferenca_tempo_s/60)%60))  #capturando minutos
    
    if len(minutos) == 1:
        minutos = '0' + minutos
    
    fdzão['DURAÇÃO'][i] = horas + ':' + minutos
    
fdzão['DURAÇÃO'].unique()

fd = fdzão[['nome','data','unidade','ENTRADA','SAIDA','DURAÇÃO','setor']]

In [32]:
print(df['extra'].value_counts())
#df['extra'].fillna('DESLOCADO', inplace = True)
print(df['extra'].value_counts())

Series([], Name: extra, dtype: int64)
Series([], Name: extra, dtype: int64)


In [33]:
df['extra'] = ''

In [34]:
df['sem_ou_fds'] = 'semana'

In [35]:
df = pd.concat([df, fd])
df = df.reset_index(drop=True)
len(df)

11

# MATANDO O RESTO:

In [36]:
H=0
print(len(df))

11


In [37]:
for h in range(H, len(df)):
    print(h)
    #click criar novo plantão
    browser.find_element("xpath", '//*[@id="shift-monthly"]/div[2]/table/tbody/tr[1]/td[4]/button').click()
    
    #esta disponivel o botão de setor
    WebDriverWait(browser, 10).until(visibility_of(browser.find_element("xpath", '//*[@id="s2id_ddlServiceSector"]/a/span[1]')))
    #Setor====
    print('base:', df['unidade'][h], '-', df['setor'][h])
    browser.find_element("xpath", '//*[@id="s2id_ddlServiceSector"]/a/span[1]').click()
    ind = scrap_unidades.loc[(scrap_unidades['unidade'] == df['unidade'][h]) & 
                             (scrap_unidades['momento'] == df['setor'][h])].index.values[0]
    unidade = scrap_unidades['unidade_count'][ind]
    setorzin = scrap_unidades['plt_count'][ind]
    print('scraping:', scrap_unidades['unidade'][ind], '-', scrap_unidades['momento'][ind])
    #SELECIONANDO Setor ##//*[@id="select2-drop"]/ul/li[unidade]/ul/li[momento]/div
    print('caixa:', browser.find_element("xpath", '//*[@id="select2-drop"]/ul/li['+str(unidade)+']/div').text, '-',
          browser.find_element("xpath", '//*[@id="select2-drop"]/ul/li['+str(unidade)+']/ul/li['+str(setorzin)+']/div').text)
    browser.find_element("xpath", '//*[@id="select2-drop"]/ul/li['+str(unidade)+']/ul/li['+str(setorzin)+']/div').click()
    print('selecionado:', browser.find_element("xpath", '//*[@id="s2id_ddlServiceSector"]/a/span[1]').text, '\n')
    
    #PROFISSIONAL
    #esta disponivel o botão de profissional
    WebDriverWait(browser, 10).until(visibility_of(browser.find_element("xpath", '//*[@id="service-ckb-list-all-owner"]')))
    
    #listar todos
    browser.find_element("xpath", '//*[@id="service-ckb-list-all-owner"]').click()
    
    browser.find_element("xpath", '//*[@id="service-ckb-list-all-owner"]').click() #########
    browser.find_element("xpath", '//*[@id="service-ckb-list-all-owner"]').click() #########
    sleep(0.5)
    browser.find_element("xpath", '//*[@id="s2id_ddlProfessionalA"]/a/span[1]').click()
    browser.find_element("xpath", '//*[@id="select2-drop"]/div/input').send_keys(df['nome'][h])
    #Dando enter
    browser.find_element("xpath", '//*[@id="select2-drop"]/div/input').send_keys(Keys.RETURN)
    #info
    print('base:', df['nome'][h])
    print('selecionado:', browser.find_element("xpath", '//*[@id="s2id_ddlProfessionalA"]/a/span[1]').text)
    print(browser.find_element("xpath", '//*[@id="service-ckb-list-all-owner"]').text)
    
    sleep(2)
    #Data
    browser.find_element("xpath", '//*[@id="modal-add-service"]/div/div[2]/div[3]/div[5]/div[1]/div[2]/div/div[2]/div[1]/div[2]/input[1]').clear()
    sleep(1)
    browser.find_element("xpath", '//*[@id="modal-add-service"]/div/div[2]/div[3]/div[5]/div[1]/div[2]/div/div[2]/div[1]/div[2]/input[1]').send_keys(df['data'][h])
    sleep(0.5)
    print(df['data'][h])
    
    #Inicio do plantão
    browser.find_element("xpath", '//*[@id="txtServiceStartTime"]').clear()
    browser.find_element("xpath", '//*[@id="txtServiceStartTime"]').send_keys(df['ENTRADA'][h])
    sleep(1)
    print('hora de entrada:', df['ENTRADA'][h], '\nhora de saida:', df['SAIDA'][h])
    #Duração do plantão
    browser.find_element("xpath", '//*[@id="txtServiceDuration"]').clear()
    browser.find_element("xpath", '//*[@id="txtServiceDuration"]').send_keys(df['DURAÇÃO'][h])
    print('duração do plantão', df['DURAÇÃO'][h])
        
    ##TIPO
    '''
    //*[@id="select2-drop"]/ul/li[3]/div
    1 - Normal
    2 - DESLOCAMENTO
    3 - VOLUNTÁRIO
    4 - COBERTURA
    '''
    if df['extra'][h] == 'extra':
        print('EXTRA')
        sleep(1)
        browser.find_element("xpath", '//*[@id="s2id_add-service-service-types"]/a/span[1]').click()
        browser.find_element("xpath", '//*[@id="s2id_add-service-service-types"]/a/span[1]').click()
        sleep(1)
        #selecionando o tipo voluntario:
        browser.find_element("xpath", '//*[@id="select2-drop"]/ul/li[3]/div').click()
        if int(df['DURAÇÃO'][h][:2]) >= 5 and df['sem_ou_fds'][h] == "semana":
            browser.find_element("xpath", '//*[@id="txtServiceValue"]').clear()
            browser.find_element("xpath", '//*[@id="txtServiceValue"]').send_keys(200) #Pontos
            print('duração >5 -> 200 pontos')
        else:
            pass
    elif df['extra'][h] == 'DESLOCADO':
        print('DESLOCADO')
        sleep(1)
        browser.find_element("xpath", '//*[@id="s2id_add-service-service-types"]/a/span[1]').click()
        browser.find_element("xpath", '//*[@id="s2id_add-service-service-types"]/a/span[1]').click()
        sleep(1)
        #selecionando o tipo voluntario:
        browser.find_element("xpath", '//*[@id="select2-drop"]/ul/li[2]/div').click()
        if int(df['DURAÇÃO'][h][:2]) >= 5 and df['sem_ou_fds'][h] == "semana":
            browser.find_element("xpath", '//*[@id="txtServiceValue"]').clear()
            browser.find_element("xpath", '//*[@id="txtServiceValue"]').send_keys(100) #Pontos
            print('duração >5 -> 100 pontos')
        else:
            pass
        sleep(2)
    else:
        if int(df['DURAÇÃO'][h][:2]) >= 5 and df['sem_ou_fds'][h] == "semana":
            browser.find_element("xpath", '//*[@id="txtServiceValue"]').clear()
            browser.find_element("xpath", '//*[@id="txtServiceValue"]').send_keys(100) #Pontos
            
            browser.find_element("xpath", '//*[@id="txtServiceValue"]').clear()
            browser.find_element("xpath", '//*[@id="txtServiceValue"]').send_keys(100) #Pontos
            print('duração >5 -> 100 pontos')
        else:
            pass
        sleep(2)
        
    #salvar plantão
    sleep(1)
    browser.find_element("xpath", '//*[@id="modal-add-service"]/div/div[1]/div[2]/button[4]').click()
    browser.find_element("xpath", '//*[@id="modal-add-service"]/div/div[1]/div[2]/button[4]').click()
    browser.find_element("xpath", '//*[@id="modal-add-service"]/div/div[1]/div[2]/button[4]').click()
    sleep(4)

    #VERIFICANDO SE HOUVE ERRO - HÁ PLANTÃO NO MESMO HORARIO
    if browser.find_element("xpath", '/html/body/div[5]/div/div[2]/div[2]/div/label').text != '':
        if df['PERIODO'][h] == '6h':
            browser.find_element("xpath", '//*[@id="ckb-add-service-ignore-conflict"]').click()#estou ciente
            sleep(1)
            browser.find_element("xpath", '//*[@id="modal-add-service"]/div/div[1]/div[2]/button[4]').click() #salvar plantão
            browser.find_element("xpath", '//*[@id="modal-add-service"]/div/div[1]/div[2]/button[4]').click()
            browser.find_element("xpath", '//*[@id="modal-add-service"]/div/div[1]/div[2]/button[4]').click()        
        else:
            browser.find_element("xpath", '//*[@id="modal-add-service"]/div/div[1]/button').click() #FECHAR
            df.loc[int(h), 'controle'] = "PROBS"
            print("PROB")
    else:
        df.loc[int(h), 'controle'] = "NORM"
    
    #browser.find_element("xpath", '//*[@id="modal-add-service"]/div/div[1]/button').click() #fechar {X}

    #não esta disponivel o X
    
    WebDriverWait(browser, 10).until_not(visibility_of(browser.find_element("xpath", '//*[@id="modal-add-service"]/div/div[1]/button')))
    H += 1
    print('='*40, '\n')
    
    if H % 5 == 0:
        print('='*80)
        print(df['controle'].value_counts())
        print('='*80)
        
print("="*50, 'TERMINOU', "="*50)
print('Total  ', len(df))
print(df['controle'].value_counts())

0
base: JABAQUARA - SEMANA - TARDE - C.CIRURGICO
scraping: JABAQUARA - SEMANA - TARDE - C.CIRURGICO
caixa: Jabaquara - Semana - Tarde - C.Cirúrgico
selecionado: Semana - Tarde - C.Cirúrgico 

base: DEBORA PHILIPPI BRESSANE
selecionado: DÉBORA PHILIPPI BRESSANE

29/09/2023
hora de entrada: 13:00 
hora de saida: 17:00
duração do plantão 04:00

1
base: OSASCO - SEMANA - TARDE - C.CIRURGICO
scraping: OSASCO - SEMANA - TARDE - C.CIRURGICO
caixa: Osasco - Semana - Tarde - C.Cirúrgico
selecionado: Semana - Tarde - C.Cirúrgico 

base: GUILHERME DE ALMEIDA COSTA
selecionado: GUILHERME DE ALMEIDA COSTA

29/09/2023
hora de entrada: 13:00 
hora de saida: 13:30
duração do plantão 00:30

2
base: ALPHA - SEMANA - MANHA - C.CIRURGICO
scraping: ALPHA - SEMANA - MANHA - C.CIRURGICO
caixa: Alpha - Semana - Manhã - C.Cirúrgico
selecionado: Semana - Manhã - C.Cirúrgico 

base: MARCELA APARECIDA CORREA MARTINS
selecionado: MARCELA APARECIDA CORREA MARTINS

29/09/2023
hora de entrada: 07:00 
hora de saida: 1

In [43]:
df["setor"][H] = "SEMANA - MANHA - C.CIRURGICO"

In [60]:
df['setor'][df['unidade'] == 'AVICCENA'].unique()
#df[(df['unidade'] == 'AVICCENA') & (df['setor'] == 'SEMANA - MANHA - COBERTURA EXTRA')]

df['setor'][(df['unidade'] == 'AVICCENA') & (df['setor'] == 'SEMANA - MANHA - COBERTURA EXTRA')] = 'SEMANA - MANHA - C.CIRURGICO'

In [63]:
df['setor'][h] = 'SEMANA - MANHA - C.CIRURGICO'

In [44]:
browser.find_element("xpath", '/html/body/div[5]/div/div[2]/div[2]/div/label').text

'Estou ciente e quero salvar mesmo assim'

In [83]:
df['setor'][df['setor'] == df['setor'][h]] = 'SEMANA - MANHA - C.CIRURGICO'

In [84]:
ind = scrap_unidades.loc[(scrap_unidades['unidade'] == df['unidade'][h]) & 
                        (scrap_unidades['momento'] == df['setor'][h])].index.values[0]

ind

182

In [46]:
df['controle'][H] = 'PROBS'
H += 1

In [38]:
df_prob = pd.concat([df_prob, df[df['controle'] != 'NORM']]).reset_index(drop=True)
print(len(df_prob))
df = df[df['controle'] == 'NORM'].reset_index(drop=True)
print(len(df))

0
11


In [39]:
def salvando_nome_arquivo(nome_arquivo, dataframe):
    if os.path.exists(nome_arquivo): #O nome_arquivo existe
        df1 = pd.read_excel(nome_arquivo)
        dataframe = pd.concat([df1, dataframe]).reset_index(drop=True)
        dataframe.to_excel(nome_arquivo, index=False)
        print("+=", len(df1), '; len =', len(dataframe)) 

    else: #O nome_arquivo não existe
        dataframe.to_excel(nome_arquivo, index=False)
        print("len =", len(dataframe)) 
        
os.remove("df_giov2.xlsx")
salvando_nome_arquivo('Feitos.xlsx', df)
print('.')
salvando_nome_arquivo('Não Feitos.xlsx', df_prob)

+= 2921 ; len = 2932
.
+= 19 ; len = 19


In [40]:
browser.quit()